In [4]:
import folium
import folium.plugins
import pandas as pd
import geopandas as gpd
import requests

#INFORMACIÓN UTILIZADA
#https://www.efrainmaps.es/english-version/free-downloads/world/
#https://www.kaggle.com/datasets/karnikakapoor/wonders-of-world?select=wonders_of_world.csv
#https://restcountries.com/v3.1/name/peru

In [5]:
#CARGO EL CSV DE LOS LUGARES IMPORTANTES

df = pd.read_csv("datasets/wonders_of_world.csv")
df.drop_duplicates(subset = ["Name"], keep='last', inplace = True)
df.loc[(df["Name"] == "Victoria Falls"), "Location"] = "Zambia"
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

#AÑADO NUEVOS CAMPOS; EN EL FUTURO DETERMINARAN LOS ESTILOS
df["icon"] = "map-marker"
df["color"] = "darkblue"
df["iconcolor"] = "#FFFFFF"

In [6]:
#CARGOS LOS .shp DE LOS PAISES
df_paises = gpd.read_file('datasets/World_Countries.shp')

In [7]:
#CORRCCION DE ERRORES DEL CSV; MUCHOS DE LOS DATOS ESTAN MAL INTRODUCIDOS
arrayPaises = []
for i in range(len(df["Name"])):   
    paises = df["Location"][i].split(", ")
    arrayPaises.append(paises[len(paises)-1].strip())


for i in range(len(arrayPaises)):
    if arrayPaises[i] == "Canada.":      
        arrayPaises[i] = "Canada"
    if arrayPaises[i] == "England":
        arrayPaises[i] = "United Kingdom"
    if arrayPaises[i] == "English Channel":
        arrayPaises[i] = "United Kingdom"
    if arrayPaises[i] == "Hawaii ,United States":
        arrayPaises[i] = "United States"
    if arrayPaises[i] == "Tibet":
        arrayPaises[i] = "China"

df["Paises"] = arrayPaises

In [8]:
#ESCOJEMOS LOS PAISES QUE TENGAN UN LUGAR DE INTERES EN SU TERRITORIO
df_paises["COUNTRY"]
arrayPaises2 = []
for i in range(len(df_paises["COUNTRY"])):    
    paises = df_paises["COUNTRY"][i].split(" (")
    if len(paises) == 2:
        arrayPaises2.append(paises[0])
    else:
        arrayPaises2.append(paises[0])

df_paises["COUNTRY"] = arrayPaises2

In [9]:
df_paises["Lista"] = df_paises["COUNTRY"].isin(arrayPaises)

In [10]:
df_paises = df_paises[(df_paises["Lista"] == True)]

In [11]:
#CONSULTAMOS LA API PARA DETERMINAR EL CONTINENTE DE NUESTROS PAISES
arrayPaises = df["Paises"].tolist()
#print(arrayPaises)
arrayContinentes = []
for i in range(len(arrayPaises)):
    if arrayPaises[i] != "Poles":
        url = "https://restcountries.com/v3.1/name/"+arrayPaises[i].lower()
        response = requests.get(url)
        arrayContinentes.append(response.json()[0]["region"])
    else:
        arrayContinentes.append("Poles")
df["conrinentes"] = arrayContinentes

In [12]:
arrayPaises = df_paises["COUNTRY"].tolist()
#print(arrayPaises)
arrayContinentes = []
for i in range(len(arrayPaises)):
    url = "https://restcountries.com/v3.1/name/"+arrayPaises[i].lower()
    response = requests.get(url)
    arrayContinentes.append(response.json()[0]["region"])

In [13]:
#SEPARAMOS LOS .shp POR CONTINENTES
df_paises["continente"] = arrayContinentes
df_oceania = df_paises[(df_paises["continente"] == "Oceania")]
df_africa = df_paises[(df_paises["continente"] == "Africa")]
df_asia = df_paises[(df_paises["continente"] == "Asia")]
df_america = df_paises[(df_paises["continente"] == "Americas")]
df_europa = df_paises[(df_paises["continente"] == "Europe")]

In [14]:
#ESTILO DE LOS MARKER
df.loc[(df["conrinentes"] == "Asia"), "color"] = "red"
df.loc[(df["conrinentes"] == "Europe"), "color"] = "darkblue"
df.loc[(df["conrinentes"] == "Europe"), "icon"] = "institution"
df.loc[(df["conrinentes"] == "Americas"), "color"] = "orange"
df.loc[(df["conrinentes"] == "Americas"), "icon"] = "dollar"
df.loc[(df["conrinentes"] == "Africa"), "color"] = "green"
df.loc[(df["conrinentes"] == "Oceania"), "color"] = "pink"
df.loc[(df["conrinentes"] == "Poles"), "color"] = "white"
df.loc[(df["conrinentes"] == "Poles"), "iconcolor"] = "#000000"

#EN EL POLO NORTE HACE FRIO POR LO QUE EL ICONO ES UN PINGUINO
df.loc[(df["conrinentes"] == "Poles"), "icon"] = "linux"


In [15]:
#ESTILOS DE LOS .shp
styleafrica = {'fillColor': '#228B22', 'color': '#228B22'}
styleasia = {'fillColor': '#FF0000', 'color': '#FF0000'}
styleamerica = {'fillColor': '#FF8000', 'color': '#FF8000'}
styleeuropa = {'fillColor': '#0080FF', 'color': '#0080FF'}
styleoceania = {'fillColor': '#FF0080', 'color': '#FF0080'}

In [18]:
m = folium.Map(zoom_start=15)

folium.GeoJson(df_africa, name="Africa",style_function=lambda x:styleafrica).add_to(m)
folium.GeoJson(df_asia, name="Asia",style_function=lambda x:styleasia).add_to(m)
folium.GeoJson(df_oceania, name="Oceania",style_function=lambda x:styleoceania).add_to(m)
folium.GeoJson(df_america, name="America",style_function=lambda x:styleamerica).add_to(m)
folium.GeoJson(df_europa, name="Europa",style_function=lambda x:styleeuropa).add_to(m)

marker_cluster = folium.plugins.MarkerCluster().add_to(m)
for i in range(len(df["Name"])): 
   
    folium.Marker([df["Latitude"][i],df["Longitude"][i]],popup=(f'{df["Name"][i]}'),icon=folium.Icon(color=df["color"][i],icon_color=df["iconcolor"][i], icon=df["icon"][i], prefix='fa')).add_to(marker_cluster)  
    

folium.LayerControl().add_to(m)
m

In [19]:
m.save(outfile= "index.html")